# Build a neural network

## Load data

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

# load the data
pickle_file = './data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


## Reformat the data
- data as a flat matrix [batch size x 28 x 28] $\rightarrow$ [batch size x 784],
- labels as float 1-hot encodings.

In [2]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


## Train a multinomial logistic regression
- Build a computation graph
- Run the operations on this graph

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

# run this computation and iterate

num_steps = 2001

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 17.661856
Training accuracy: 12.1%
Validation accuracy: 13.9%
Loss at step 100: 2.278361
Training accuracy: 71.8%
Validation accuracy: 71.0%
Loss at step 200: 1.838089
Training accuracy: 74.9%
Validation accuracy: 73.2%
Loss at step 300: 1.592305
Training accuracy: 76.3%
Validation accuracy: 74.0%
Loss at step 400: 1.424579
Training accuracy: 77.0%
Validation accuracy: 74.2%
Loss at step 500: 1.300651
Training accuracy: 77.9%
Validation accuracy: 74.4%
Loss at step 600: 1.204208
Training accuracy: 78.4%
Validation accuracy: 74.5%
Loss at step 700: 1.126281
Training accuracy: 79.1%
Validation accuracy: 74.6%
Loss at step 800: 1.061749
Training accuracy: 79.5%
Validation accuracy: 74.8%
Loss at step 900: 1.007432
Training accuracy: 80.0%
Validation accuracy: 74.9%
Loss at step 1000: 0.961093
Training accuracy: 80.5%
Validation accuracy: 75.0%
Loss at step 1100: 0.921049
Training accuracy: 80.8%
Validation accuracy: 75.0%
Loss at step 1200: 0.886033
Training ac

## Consider stochastic gradient 
Stochastic gradient is much faster. Create a Placeholder node which will be fed actual data at every call of ```session.run()```.

In [7]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 17.348957
Minibatch accuracy: 14.1%
Validation accuracy: 17.1%
Minibatch loss at step 500: 1.333912
Minibatch accuracy: 78.1%
Validation accuracy: 76.0%
Minibatch loss at step 1000: 1.277300
Minibatch accuracy: 75.0%
Validation accuracy: 76.8%
Minibatch loss at step 1500: 0.852153
Minibatch accuracy: 81.2%
Validation accuracy: 77.6%
Minibatch loss at step 2000: 0.906070
Minibatch accuracy: 81.2%
Validation accuracy: 78.3%
Minibatch loss at step 2500: 1.005196
Minibatch accuracy: 73.4%
Validation accuracy: 78.6%
Minibatch loss at step 3000: 1.019399
Minibatch accuracy: 75.0%
Validation accuracy: 79.0%
Minibatch loss at step 3500: 0.790643
Minibatch accuracy: 80.5%
Validation accuracy: 78.5%
Minibatch loss at step 4000: 0.713814
Minibatch accuracy: 80.5%
Validation accuracy: 79.4%
Minibatch loss at step 4500: 0.826627
Minibatch accuracy: 81.2%
Validation accuracy: 79.5%
Minibatch loss at step 5000: 0.822478
Minibatch accuracy: 81.2%
Validation accura

## Nueral network with 1 hidden layer
A 1-hidden layer neural network with rectified linear units ```nn.relu()``` and 1024 hidden nodes. This model should improve validation / test accuracy.

In [8]:
batch_size = 128
num_hidden_nodes = 1024 

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # Hidden layer
  weights_h = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_h = tf.Variable(tf.zeros([num_hidden_nodes]))

  tf_train_h = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h)+biases_h)
  tf_valid_h = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h)+biases_h)
  tf_test_h = tf.nn.relu(tf.matmul(tf_test_dataset, weights_h)+biases_h)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_h, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_h, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_h, weights) + biases)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 473.580688
Minibatch accuracy: 6.2%
Validation accuracy: 30.4%
Minibatch loss at step 500: 25.422539
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss at step 1000: 9.588213
Minibatch accuracy: 81.2%
Validation accuracy: 80.4%
Minibatch loss at step 1500: 8.829886
Minibatch accuracy: 89.8%
Validation accuracy: 80.6%
Minibatch loss at step 2000: 3.982296
Minibatch accuracy: 84.4%
Validation accuracy: 80.2%
Minibatch loss at step 2500: 2.738194
Minibatch accuracy: 82.8%
Validation accuracy: 82.3%
Minibatch loss at step 3000: 2.103939
Minibatch accuracy: 82.0%
Validation accuracy: 82.2%
Test accuracy: 89.3%
